# Assignment 1 - _Foundations of Information Retrieval '20/'21_

This assignment is divided in 3 parts, which have to be delivered all together before 04/10/2020 (strictly - no extensions will be granted!), via Canvas. Delivery of the assignment solutions is mandatory.

We will use [ElasticSearch](https://www.elastic.co/) as search engine, as it provides state-of-the-art tools to implement your own engine, and let you focus on methodological aspects of search implementation and optimization.

The assignment is about text-based Information Retrieval and it is structured in three parts:
1. IR performance evaluation (implementation of performance metrics)
2. Setting up a search engine, pre-processing and indexing using ElasticSearch (Indexing, Analyzers)
3. Implementation and optimization of a model of search using ElasticSearch (Similarity measures)


This assignment file contains exercises, marked with the section title __Exercise 01.(x)__, which are evaluated, and other sections that contain support code which you should use as it is. Write your answers between the comments `BEGIN ANSWER` and `END ANSWER`.
Try to complete the solutions for all the exercise sections. 

_Note:_ we leave the comment `#THIS IS GRADED!` in the sections that will be considered for evauation and grading.


### Initial preparation (self-study)
For the It is good to acquire basic knowledge of Python (or refresh it a bit).
For the second and third part of the assignment, please study yourself the [Getting Started guide](https://www.elastic.co/guide/en/elasticsearch/reference/current/getting-started.html)" of ElasicSearch and get acquainted with the framework.


# PART 01 - Performance evaluation


### Background information and reading
To solve the exercises in Part 01, study the slides of Lecture 01 (available on Canvas) and the reference book chapter (Christopher D. Manning, Prabhakar Raghavan and Hinrich Schütze, Introduction to Information Retrieval, [Chapter 8, Evaluation in information retrieval](http://nlp.stanford.edu/IR-book/pdf/08eval.pdf), Cambridge University Press. 2008)

### Basic concepts
Suppose the set of relevant documents (the document identifiers - _doc-IDs_) is called `relevant`, then we might define those as follows (in Python):

In [2]:
relevant = set([2, 3, 5, 8, 13, 21])

A perfect run would retrieve exactly these 6 documents in any order. Now, suppose the list of retrieved documents (the document identifiers - _doc-IDs_) is called `retrieved`, and contains the following _doc-IDs_:

In [3]:
retrieved = [4, 2, 18, 16, 8, 46, 32, 22, 47, 39, 3]

One of the simplest evaluation measures we can think of is the _Success at rank 1_. The measure answers the question: Was the first document retrieved a relevant document? _Success at rank 1_ returns 1 if the first document is relevant, and 0 otherwise. A possible implementation is: 

In [4]:
def success_at_1 (relevant, retrieved):
    if len(retrieved) > 0 and retrieved[0] in relevant:
        return 1
    else:
        return 0

success_at_1(relevant, retrieved)

0

The first retrieved documentid is 4 which is not in the set of relevant documents, so the score is 0.

Note how easy it is to check if an item occurs in a Python set or list by using the keyword: `in`. Similarly, you can loop over all items in a set of list with: 
`for doc in retrieved:`, 
where doc will refer to each item in the set or list. 

Be sure to use the internet to sharpen your knowledge about Python constructs, for instance on [Python list slicing](https://duckduckgo.com/?q=python+list+slicing). Also note that the code above checks if at least one document is retrieved to avoid an index out of bounds exception (i.e. we avoid to access an empty vector).

> __Suggestion: to be sure of the correctness of the implementations of the performance metrics you are requested, you can compute their values manually and compare them with those of your functions. This is important, as you will use these metrics for later exercises and to compare different models.__

## Exercise 01.A: _Success at k_
The measure _Success at k_ returns 1 if a relevant document is among the first _k_ documents retrieved and zero otherwise. Implement _Success at 5_ below.

> Success at _k_ measures are well-suited in cases where there is typically only one relevant document (or retrieving one relevant document is enough).

In [5]:
#THIS IS GRADED!


def success_at_5(relevant, retrieved):
    # BEGIN ANSWER
    for k in retrieved[:5]:
        if k in relevant:
            return 1        
    return 0
    # END ANSWER
    
success_at_5(relevant, retrieved)

1

Similarly implement success at rank 10

In [6]:
#THIS IS GRADED!

def success_at_10(relevant, retrieved):
    # BEGIN ANSWER   
    for k in retrieved[:10]:
        if k in relevant:
            return 1   
    return 0
    # END ANSWER
    
success_at_10(relevant, retrieved)

1

## Exercise 01.B: _Precision, Recall and F-measure_
Implement _Precision_ using Formula 8.1 from [Manning, Raghavan and Schütze](http://nlp.stanford.edu/IR-book).

_Hint:_ one can count the number of documents in a list by using the built-in Python function [len()](https://docs.python.org/3/library/functions.html#len) (e.g. `len(retrieved)` for the number of retrieved documents). 

In [7]:
#THIS IS GRADED!

def precision(relevant, retrieved):
    # BEGIN ANSWER
    if not retrieved:
        return 1
    
    relevant_and_retrieved = [k for k in retrieved if k in relevant]
    return len(relevant_and_retrieved) / len(retrieved)
    # END ANSWER
    
precision(relevant, retrieved)

0.2727272727272727

Implement _Recall_ using Formula 8.2 from [Manning, Raghavan and Schütze](http://nlp.stanford.edu/IR-book).

In [8]:
#THIS IS GRADED!

def recall(relevant, retrieved):
    # BEGIN ANSWER
    if not relevant:
        return 1
    
    relevant_and_retrieved = [k for k in retrieved if k in relevant]
    return len(relevant_and_retrieved) / len(relevant)
    # END ANSWER
    
recall(relevant, retrieved)

0.5

The balanced F measure (_F_ with β=1) is defined as the harmonic mean of precision and
recall. Implement _F_ using Formula 8.6 from [Manning, Raghavan and Schütze](http://nlp.stanford.edu/IR-book).

> Tip: reuse your implementations of precision and recall

In [9]:
#THIS IS GRADED!

def f_measure(relevant, retrieved):
    # BEGIN ANSWER
    P = precision(relevant, retrieved)
    R = recall(relevant, retrieved)
    return 2*P*R/(P+R)
    # END ANSWER
    
f_measure(relevant, retrieved)

0.3529411764705882

## Exercise 01.C: _Precision at rank k_ and  _R-Precision_

Precision, Recall and F are _set_-based measures and suited for unranked lists of documents. If our search system returns a ranked _list_ of results, we can measure precision for several cut-off levels _k_ in the ranked list, i.e. we evaluate the relevance of the TOP-_k_ retrieved documents (see lecture slides and the related book chapter). 
We did this before with the _Success at rank 5_ measure for _k_=5.

Implement below the function `precision_at_k()` that measures the precision at rank _k_

> Interesting fact: For _k_=1, the _Precision at rank 1_ would be the samen as _Success at rank 1_ (why?) - Because it must be either 1 out of 1 right or 0 out of 1 correct so. Therefore the precision must be 1 or 0.

In [10]:
#THIS IS GRADED!

def precision_at_k(relevant, retrieved, k):
    # BEGIN ANSWER
    return precision(relevant, retrieved[:k])
    # END ANSWER
    
precision_at_k(relevant, retrieved, k=1)

0.0

Implement R-Precision (function `r_precision()`) as defined on Page 161 of [Manning, Raghavan and Schütze](http://nlp.stanford.edu/IR-book).

In [11]:
#THIS IS GRADED!

def r_precision(relevant, retrieved):
    # BEGIN ANSWER
    k = len(relevant)
    return precision(relevant, retrieved[:k])
    # END ANSWER
    
r_precision(relevant, retrieved)

0.3333333333333333

## Exercise 01.D:  Interpolated precision at _recall_ X

Another way to address ranked retrieval is to measure precision for several _recall_ levels _X_.

Implement the function `interpolated_precision_at_recall_X()` that measures the interpolated precision at recall level _X_ as defined by Formula 8.7 of [Manning, Raghavan and Schütze](http://nlp.stanford.edu/IR-book).

> Tip: calculate for each rank the recall. If the recall is greater than or equal to X, 
> calculate the precision. Keep the highest (maximum) precision of those to be returned at the end.

In [12]:
#THIS IS GRADED!

def interpolated_precision_at_recall_X (relevant, retrieved, X):
    # BEGIN ANSWER
    # The interpolated precision at recall X is undefined where the max recall for the retrieved set does not reach X.
    if recall(relevant, retrieved) < X:
        return 0
    
    P = 0    
    # Loop through each rank.
    for k, _ in enumerate(retrieved):
        if recall(relevant, retrieved[:k]) >= X:
            P = max(P, precision_at_k(relevant, retrieved, k))
    
    return P
    # END ANSWER
    
interpolated_precision_at_recall_X(relevant, retrieved, X=0.1) 

0.5

## Exercise 01.E:  _Average Precision_

For a single information need, _Average Precision_ is the average of the precision value obtained for the set of top k documents existing after each relevant document is retrieved (see [Manning, Raghavan and Schütze](http://nlp.stanford.edu/IR-book), Pages 159 and 160). Implement _Average Precision_ for a single information need. 

In [13]:
#THIS IS GRADED!

def average_precision(relevant, retrieved):
    # BEGIN ANSWER
    # Initialise list of precisions with a zero for each relevant document.
    P = [0] * len(relevant)
    
    for i, doc in enumerate(relevant):
        # If a relevant document is not retrieved, the precision value is taken to be zero. 
        if doc not in retrieved:
            P[i] = 0
        else:
            # Find the precision for the top k documents when doc is retrieved.
            k = retrieved.index(doc)
            P[i] = precision_at_k(relevant, retrieved, k)
    
    # Return the average precision
    return sum(P)/len(P)
    # END ANSWER

average_precision(relevant, retrieved)

0.075

## Measures in TREC 

The relevance judgments are provided by TREC in so-called _"qrels"_ files that look as follows:

    1000 Q0 1341 1
    1000 Q0 1231 0
    1001 Q0 12332 1
     ...

The first column is the query identifier, while the second column is the query number within that topic (it is currently unused and should always be Q0). The third column is the document identifier that was examined by the judges. The fourth column is the relevance of the document (_1_ means the document was relevant and _0_ means the document was not relevant).

Below we provide some Python code that reads the _qrels_ and the _run_. The qrels will be put in the Python dictionary `all_relevant`. A [Python dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) provides quick lookup of a set of values given a key. We will use the `query_id` as a key, and a [set](https://docs.python.org/3/tutorial/datastructures.html#sets) of relevant document identifiers. For the partial qrels file above, `all_relevant` would look as follows:

    {
        "1000": set(["1341", "1231"]),
        "1001": set(["12332"])
    }
    
We will use a dictionary called `all_retrieved` with `query_id` as key, and as value a [Python list](https://docs.python.org/3/tutorial/introduction.html#lists) of document identifiers retrieved by the IR system:

    {
        "1000": ["1341", "12346, "2345"],
        "1001": [..., ..., ...],
        ...
    }

Note that, with this data structure, for each `query_id` we can easily access the list of retrieved and relevant documents, and compute the performance metrics. We can then average these measures over all the queries to compute the mean performance of the IR system on the given retrieval task.

Please examine the code below, and make sure you understand every line.

In [14]:
def read_qrels_file(qrels_file):  # reads the content of he qrels file
    trec_relevant = dict()  # query_id -> set([docid1, docid2, ...])
    with open(qrels_file, 'r') as qrels:
        for line in qrels:
            (qid, q0, doc_id, rel) = line.strip().split()
            if qid not in trec_relevant:
                trec_relevant[qid] = set()
            if (rel == "1"):
                trec_relevant[qid].add(doc_id)
    return trec_relevant

def read_run_file(run_file):  
    # read the content of the run file produced by our IR system 
    # (in the following exercises you will create your own run_files)
    trec_retrieved = dict()  # query_id -> [docid1, docid2, ...]
    with open(run_file, 'r') as run:
        for line in run:
            (qid, q0, doc_id, rank, score, tag) = line.strip().split()
            if qid not in trec_retrieved:
                trec_retrieved[qid] = []
            trec_retrieved[qid].append(doc_id) 
    return trec_retrieved
    

def read_eval_files(qrels_file, run_file):
    return read_qrels_file(qrels_file), read_run_file(run_file)

(all_relevant, all_retrieved) = read_eval_files('data/training-qrels.txt', 'data/baselineTREC.run')

### Exercise 01.F: _number of queries_ and _number of retrieved documents per query_
 
Write the Python code that counts the number of queries in the file `baseline.run` and print the value (use the result from the cell above). 

_Hint:_ print the structure and content of the `all_retrieved` and `all_relevant` data structures to understand them better.

In [15]:
#THIS IS GRADED!

# BEGIN ANSWER

# baselineTREC.run is read into the dict all_retrieved. By finding the length, we get the number of keys (queries)
num_queries = len(all_retrieved)
print(num_queries)

# END ANSWER

50


Write the code that counts, for each query in your baseline run, the number of documents that were retrieved for that query (use `print()` to print the result for each `query_id`).

In [16]:
#THIS IS GRADED!

# BEGIN ANSWER
for query in all_retrieved:
    num_documents = len(all_retrieved[query])
    print("Query: ", query, "  # Documents Retrieved: ", num_documents)
# END ANSWER

Query:  1   # Documents Retrieved:  10
Query:  2   # Documents Retrieved:  10
Query:  3   # Documents Retrieved:  10
Query:  4   # Documents Retrieved:  10
Query:  5   # Documents Retrieved:  10
Query:  6   # Documents Retrieved:  10
Query:  7   # Documents Retrieved:  10
Query:  8   # Documents Retrieved:  10
Query:  9   # Documents Retrieved:  10
Query:  10   # Documents Retrieved:  10
Query:  11   # Documents Retrieved:  10
Query:  12   # Documents Retrieved:  10
Query:  13   # Documents Retrieved:  10
Query:  14   # Documents Retrieved:  10
Query:  15   # Documents Retrieved:  10
Query:  16   # Documents Retrieved:  10
Query:  17   # Documents Retrieved:  10
Query:  18   # Documents Retrieved:  10
Query:  19   # Documents Retrieved:  10
Query:  20   # Documents Retrieved:  10
Query:  21   # Documents Retrieved:  10
Query:  22   # Documents Retrieved:  10
Query:  23   # Documents Retrieved:  10
Query:  24   # Documents Retrieved:  10
Query:  25   # Documents Retrieved:  10
Query:  2

## Exercise 01.G: _mean average precision_
Using the `average_precision()` function you implemented above, write the code to compute the _Mean Average Precision_ for the `baseline.run` results. 

In [17]:
#THIS IS GRADED!

def mean_average_precision(all_relevant, all_retrieved):
    # BEGIN ANSWER
    
    count = len(all_retrieved)
        
    precision_per_query = [average_precision(all_relevant[query], all_retrieved[query])  for query in all_retrieved]
    total = sum(precision_per_query)
    
    # END ANSWER
    return "mean AP: ", total / count

mean_average_precision(all_relevant, all_retrieved)

('mean AP: ', 0.05616925010473398)

## TREC evaluation

Below you find a function that take `all_relevant` and `all_retrieved` to compute the mean result. It computes the mean value over all queries. The function `mean_metric()`'s first function argument, `measure`, is a special argument: it is a function too! The `mean_metric` function sums the total score for the particular measure and divides it by the number of queries. It computes the average measures over all the queries' results.

_This part will be reused later to compare the results of different models._

In [18]:
def mean_metric(measure, all_relevant, all_retrieved):
    total = 0
    count = 0
    for qid in all_relevant:
        relevant  = all_relevant[qid]
        retrieved = all_retrieved.get(qid, [])
        value = measure(relevant, retrieved)
        total += value
        count += 1
    return "mean " + measure.__name__, total / count

# Example of use of the mean_metric function: computing the average r_precision
mean_metric(r_precision, all_relevant, all_retrieved)

('mean r_precision', 0.0769047619047619)

### TREC overview of the results
The following two functions use your implementation of the metrics to create an evaluation overview of the TREC benchmark data. Give a look at the numbers and make you own interpretations of the results. 

In [19]:
def trec_eval(qrels_file, run_file):

    def precision_at_1(rel, ret): return precision_at_k(rel, ret, k=1)
    def precision_at_5(rel, ret): return precision_at_k(rel, ret, k=5)
    def precision_at_10(rel, ret): return precision_at_k(rel, ret, k=10)
    def precision_at_50(rel, ret): return precision_at_k(rel, ret, k=50)
    def precision_at_100(rel, ret): return precision_at_k(rel, ret, k=100)
    def precision_at_recall_00(rel, ret): return interpolated_precision_at_recall_X(rel, ret, X=0.0)
    def precision_at_recall_01(rel, ret): return interpolated_precision_at_recall_X(rel, ret, X=0.1)
    def precision_at_recall_02(rel, ret): return interpolated_precision_at_recall_X(rel, ret, X=0.2)
    def precision_at_recall_03(rel, ret): return interpolated_precision_at_recall_X(rel, ret, X=0.3)
    def precision_at_recall_04(rel, ret): return interpolated_precision_at_recall_X(rel, ret, X=0.4)
    def precision_at_recall_05(rel, ret): return interpolated_precision_at_recall_X(rel, ret, X=0.5)
    def precision_at_recall_06(rel, ret): return interpolated_precision_at_recall_X(rel, ret, X=0.6)
    def precision_at_recall_07(rel, ret): return interpolated_precision_at_recall_X(rel, ret, X=0.7)
    def precision_at_recall_08(rel, ret): return interpolated_precision_at_recall_X(rel, ret, X=0.8)
    def precision_at_recall_09(rel, ret): return interpolated_precision_at_recall_X(rel, ret, X=0.9)
    def precision_at_recall_10(rel, ret): return interpolated_precision_at_recall_X(rel, ret, X=1.0)

    (all_relevant, all_retrieved) = read_eval_files(qrels_file, run_file)
    
    unknown_qids = set(all_retrieved.keys()).difference(all_relevant.keys())
    if len(unknown_qids) > 0:
        raise ValueError("Unknown qids in run: {}".format(sorted(list(unknown_qids))))

    metrics = [success_at_1,
               success_at_5,
               success_at_10,
               r_precision,
               precision_at_1,
               precision_at_5,
               precision_at_10,
               precision_at_50,
               precision_at_100,
               precision_at_recall_00,
               precision_at_recall_01,
               precision_at_recall_02,
               precision_at_recall_03,
               precision_at_recall_04,
               precision_at_recall_05,
               precision_at_recall_06,
               precision_at_recall_07,
               precision_at_recall_08,
               precision_at_recall_09,
               precision_at_recall_10,
               average_precision]

    return [mean_metric(metric, all_relevant, all_retrieved) for metric in metrics]


def print_trec_eval(qrels_file, run_file):
    results = trec_eval(qrels_file, run_file)
    print("Results for {}".format(run_file))
    for (metric, score) in results:
        print("{:<30} {:.4}".format(metric, score))

print_trec_eval('data/training-qrels.txt', 'data/baselineTREC.run')

Results for data/baselineTREC.run
mean success_at_1              0.16
mean success_at_5              0.24
mean success_at_10             0.3
mean r_precision               0.0769
mean precision_at_1            0.16
mean precision_at_5            0.08
mean precision_at_10           0.052
mean precision_at_50           0.052
mean precision_at_100          0.052
mean precision_at_recall_00    1.0
mean precision_at_recall_01    0.1747
mean precision_at_recall_02    0.1047
mean precision_at_recall_03    0.08028
mean precision_at_recall_04    0.08028
mean precision_at_recall_05    0.08028
mean precision_at_recall_06    0.0425
mean precision_at_recall_07    0.02917
mean precision_at_recall_08    0.02917
mean precision_at_recall_09    0.02917
mean precision_at_recall_10    0.02917
mean average_precision         0.05617


## Exercise 01.H: _significance testing_

Testing the statistical significance of differences in the results of different IR systems is important (see slides and course book - Section 8.8). One of the basic tests one can perform is the two-tailed [sign test](https://en.wikipedia.org/wiki/Sign_test).


For this exercise, we use the run files obtained by  [Hiemstra and Aly](https://djoerdhiemstra.com/wp-content/uploads/trec2014mirex-draft.pdf) for TREC 2014. The `utbase.run` file was generated usinf Language Modeling, while `utexact.run` was generated using an IR system based on mathing the exact query string, abd ranking the documents by  the number of exact matches found. The exact run improves the _Precision at 5_ to 0.456 (compared to 0.440 for the baseline run).  

Implement the code to perform the _sign test_ of statistical significance.
_Hint:_ for each sign, compute the number of queries that increase/descrease performance (called `better, worse` in the code below). How would you use these values to compute the _p_ value of the two-tailed sign test? Is the difference between _utbase_ and _utexact_ significant?
    
Answer: Conduct a binomial test where `better` is the number of successes, `worse` is the number of failures, and the null hypothesis assumes a binomial distribution with p = 0.5. 
Since the performance of the second method is better for 9 queries and also worse for 9 queries, then we get a p-value of 1.0 and fail to reject the null hypothesis. i.e. the difference between _utbase_ and _utexact_ is not statistically significant.

In [20]:
#THIS IS GRADED!

def sign_test_values(measure, qrels_file, run_file_1, run_file_2):
    all_relevant = read_qrels_file(qrels_file)
    all_retrieved_1 = read_run_file(run_file_1)
    all_retrieved_2 = read_run_file(run_file_2)
    better = 0
    worse  = 0
    # BEGIN ANSWER
    
    for query in all_retrieved_1:
        performance_1 = measure(all_relevant[query], all_retrieved_1[query])
        performance_2 = measure(all_relevant[query], all_retrieved_2[query])
        
        if performance_2 > performance_1:
            better += 1
        # Exclude queries with no performance difference between the two methods.
        elif performance_2 < performance_1:
            worse += 1
    
    # END ANSWER
    return(better, worse)
    
def precision_at_rank_5(rel, ret):
    return precision_at_k(rel, ret, k=5)

sign_test_values(precision_at_rank_5, 'data/trec.qrels', 'data/utbase.run', 'data/utexact.run')

# from scipy.stats import binom_test
# w = sign_test_values(precision_at_rank_5, 'data/trec.qrels', 'data/utbase.run', 'data/utexact.run')
# binom_test(w) # Accept the default arguments for the function
### Returns a p-value of 1 > 0.05, thus we fail to reject the null. i.e. there is no difference in performance between the two methods.


(9, 9)

# Part 02 - Indexing and querying with ElasticSearch

### Preparation: Getting started with Elasticsearch

We strongly advice you to go through the "Elasticsearch, [reference guide](https://www.elastic.co/guide/en/elasticsearch/reference/current/getting-started.html)", and work on the tutorials. The following parts of the assignment will be based on ElasticSearch.

You can skip the section on [Installation](https://www.elastic.co/guide/en/elasticsearch/reference/current/_installation.html), as we provide it already installed in the Virtual Machine.

> If you want (disclaimer: we do __not__ give help with this!), you can 
> follow the [Installation](https://www.elastic.co/guide/en/elasticsearch/reference/current/_installation.html) to run Elasticsearch on your laptop without VM. But beware, your system will now be different from the 
> ones of your colleagues and they might not be able to help you if 
> you have problems that are specific to your system, your operating
> system, or your Elasticsearch version.

### Starting/Stopping ElasticSearch
To start ElasticSearch on the virtual machine, you can type `sudo service elasticsearch start` in a Terminal.
To stop the ElasticSearch server, instead, you can type `sudo service elasticsearch stop`. Refer at the [the official guide](https://www.elastic.co/guide/en/elasticsearch/reference/current/deb.html#deb-running-init), for more information.

### The REST API

Elasticsearch runs its own server that can be accessed by a regular web browser as the client, for instance by opening this link in your browser: http://localhost:9200. 

Elasticsearch will respond with something like:

    {
        "name" : "fir-machine",
        "cluster_name" : "elasticsearch",
        "cluster_uuid" : "w7SBVo1ESVivMApbLIqRvA",
        "version" : {
            "number" : "7.9.0",
            "build_flavor" : "default",
            "build_type" : "deb",
            "build_hash" : "a479a2a7fce0389512d6a9361301708b92dff667",
            "build_date" : "2020-08-11T21:36:48.204330Z",
            "build_snapshot" : false,
            "lucene_version" : "8.6.0",
            "minimum_wire_compatibility_version" : "6.8.0",
            "minimum_index_compatibility_version" : "6.0.0-beta1"
        },
        "tagline" : "You Know, for Search"
    }


If you see this, then your Elasticsearch node is up and running. The RESTful API uses simple text or JSON over HTTP. 

> REST, API, JSON, HTTP, that's a lot of abbreviations! It is good to
> be familiar with the terminology. Let us explain: The Elasticsearch
> response is not (only) intended for humans. It is supposed to be used 
> by applications that run on the client machines, and therefore the
> interface is called an Application Programming Interface (API). The 
> API uses a format called JSON (JavaScript Object Notation), which 
> can be easily read by machines (and humans). The API sends its JSON
> response using the same method as your web browser displays web
> pages. This method is called HTTP (Hyper Text Transfer Protocol), 
> and it is the reason you can inspect the response in a normal web
> browser. APIs that use HTTP are called RESTful interfaces. REST 
> stands for REpresentational State Transfer, arguably one of the
> simplest ways to define an API.


### Kibana, cURL, and more cURL 

You can interact with your Elasticsearch service in different ways. In this first assignment we will describe three ways. Later during the practical work we will use the Python Elasticsearch client.

1. Using the Kibana Console
2. Using cURL
3. Using cURL from a Jupyter notebook (not recommended)

#### Kibana
Kibana provides a web interface to interact with your Elasticsearch service. It's available from http://localhost:5601. You can use Kibana to create interactive dashboards visualizing data in your Elasticsearch indices. It also provides a console to execute Elasticsearch commands. It's available from http://localhost:5601/app/kibana#/dev_tools

To start Kibana on the virtual machine, you can type `sudo service kibana start` in a Terminal.
To stop the Kibana server, instead, you can type `sudo service kibana stop`.

Many examples from the Elasticsearch user guide can be directly executed in Kibana by clicking the `VIEW IN CONSOLE` button.

#### cURL
[CURL](https://en.wikipedia.org/wiki/CURL) is a software tool that enables you to execute HTTP method requests from the commandline. The name originally stood for "see URL". 

Curl is already installed in the VM operating system. Let's open a bash terminal.
You can exit the shell by executing `exit`.
You can execute curl commands on this prompt, for instance retrieving the Elasticsearch state.
Note you have to use `localhost` as the hostname:
```
labs@fir-machine:~$ curl localhost:9200
{
  "name" : "epRATWu",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "KsOTBsyeTmy6fJCcZ64d_A",
  "version" : {
    "number" : "6.2.4",
    "build_hash" : "ccec39f",
    "build_date" : "2018-04-12T20:37:28.497551Z",
    "build_snapshot" : false,
    "lucene_version" : "7.2.1",
    "minimum_wire_compatibility_version" : "5.6.0",
    "minimum_index_compatibility_version" : "5.0.0"
  },
  "tagline" : "You Know, for Search"
}
```

#### cURL from this notebook

Alternatively, jupyter notebooks allow you to directly execute cURL commands (or other shell commands), by starting a line of code with an exclamation mark (see example below). Plase be warned: when executing commands which result in long output (for instance when indexing a large number of documents), stick to the terminal to execute curl commands. Jupyter might freeze when handling long output from the shell.

## Assignment Part 02 (Let's go!)

_You can work on this part after Lecture 01 already, if you want_

For the following exercises, you will use a TREC genomics document collection and queries. 
It is stored in the folder `data/` in the directory where you have been instructed to place the assignment notebooks (`/`).

The collections contains:

* `trec-medline.json` (the collection in Elasticsearch batch format - because of its size it cannot be indexed with a single curl command!)
* `training-queries-simple.txt` (test queries)
* `training-qrels.txt` (the "relevance judgements" for the test queries, i.e. the correct answers)
* `test-queries-simple.txt`
* `example_matches20.txt` (20 example matches)

To make things easy, the data is already provided in Elasticsearch' batch processing format. 
Inspect the collection file in the terminal:

`head trec-medline.json`

This shows the first 5 documents in the collection (in JSON format prepared for ElasticSearch, as you have seen in the tutorial)

## Exercise 02.A: _indexing_ and _first queries_

Execute the following cell to index the collection in an Elasticsearch index called `genomics'. This code uses the Elasticsearch python api, which we will discuss later (you can read about it yourself, in the meanwhile).

_Note:_ indexing the TREC genomics collection will take some time.

In [6]:
import elasticsearch
import elasticsearch.helpers
import json

def documents():
    """ generates the documents to be indexed as dictionaries """
    with open('data/trec-medline.json') as inp:
        while True:
            try:
                line = next(inp)  # ignore odd line nrs
                if line is None:
                    break
                try:
                    docline = next(inp)
                    doc = json.loads(docline)
                    yield doc
                except json.JSONDecodeError as e:
                    # should not occur (but ignore it anyway)
                    pass
            except StopIteration as e:
                break

In [7]:
es = elasticsearch.Elasticsearch('localhost')

# remove if it already exists
es.indices.delete(index="genomics", ignore=[400, 404])

# and bulk index it
print("Indexing documents, this will take some time...")
_ = elasticsearch.helpers.bulk(
        es, 
        documents(),
        index="genomics",
        chunk_size=2000,
        request_timeout=30
    )
print("Done")

Indexing documents, this will take some time...
Done


Query the index called Genomics and determine how many items are index. 

In [8]:
#THIS IS GRADED!

# write the code here
# BEGIN ANSWER
ind = 'genomics'
res = es.count(index=ind).get('count')

print("There are", res, "items in the index", ind)

# END ANSWER

There are 525937 items in the index genomics


Using the command line (or the Kibana console), search for all documents that contain the word `blood`. 

How many documents containing the term `blood` are there in your index? (searching all fields of the documents).

In [9]:
#THIS IS GRADED!

# write the code that generates the answer here (you may also use Kibana)
# BEGIN ANSWER
# Using Kibana console:
POST genomics/_count
{
 "query": {
   "query_string": {
     "query": "blood"
   }
 }
}
# Returns:
# {
#   "count" : 68275,
#   "_shards" : {
#     "total" : 1,
#     "successful" : 1,
#     "skipped" : 0,
#     "failed" : 0
#   }
# }
# END ANSWER

SyntaxError: invalid syntax (<ipython-input-9-a8f361d7200e>, line 6)

## Exercise 02.B: the Python ElasticSearch library

#### Preparation
The command line is fine for doing basic operations on your Elasticsearch indices, but as soon as things get more complex, you better use custom client programs.
We will use the [Elasticsearch client library for Python](https://elasticsearch-py.readthedocs.io). This library will execute the HTTP requests that you have used before (with CURL or Kibana). The library is pre-installed on the VM.

#### Exercise
Write the code that searches the index for _"blood"_ using the [search()](https://elasticsearch-py.readthedocs.io/en/master/api.html#elasticsearch.Elasticsearch.search) function. Your code will take at minimum the following steps:

1. import the python library `elasticsearch`.
2. open a connection with the Elasticsearch host `'elasticsearch'` with `Elasticsearch()`.
3. execute a search with `search()` using the index `genomics`, and a correct query body.
4. print the JSON output of Elasticsearch 

How many hits are there in your index? Is the result the same as in Exercise 01.?

> Elasticsearch runs on localhost on your laptop, at port 9200 (so as http://localhost:9200)


In [28]:
#THIS IS GRADED!

import elasticsearch

# your code below

# Open a connection with Elasticsearch host. 'es' already exists, so call this es2
es2 = elasticsearch.Elasticsearch('localhost')

cnt = es2.count(index='genomics', body = {
 "query": {
   "query_string": {
     "query": "blood"
   }
 }
})

# Get the resulting hits. search() has a maximum of counting 10,000 hits. 
# Returns the first 10 search results.
res_10 = es2.search(index='genomics', body = {
 "query": {
   "query_string": {
     "query": "blood"
   }
 }
})

# Using size=10000 allows us to retrieve the max number of docs in the query results so that we can inspect them.
res_10k = es2.search(index='genomics', size = 10000, body = {
 "query": {
   "query_string": {
     "query": "blood"
   }
 }
})

print("Total count of matching documents:", cnt['count']) # 68,275 hits = same as Exercise 1
print("\nSearch results (First 10 results. The total hits shown is capped at 10,000):\n", res_10)


Total count of matching documents: 68275

Search results (First 10 results. The total hits shown is capped at 10,000):
 {'took': 102, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 15.369452, 'hits': [{'_index': 'genomics', '_type': '_doc', '_id': 'gVAMtXQBu6mw5_RMO2jn', '_score': 15.369452, '_source': {'CON': 'J Pharmacokinet Pharmacodyn. 2001 Apr;28(2):155-69. PMID: 11381568', 'CY': 'England', 'DA': '20020826', 'DCOM': '20030312', 'DP': '2002 Feb', 'EDAT': '2002/08/27 10:00', 'IP': '1', 'IS': '1567-567X', 'JID': '101096520', 'LA': 'eng', 'LR': '20030313', 'MHDA': '2003/03/13 04:00', 'PG': '95-7; author reply 99', 'PMID': '12194538', 'PST': 'ppublish', 'PT': 'Letter', 'SB': 'IM', 'SO': 'J Pharmacokinet Pharmacodyn 2002 Feb;29(1):95-7; author reply 99.', 'TA': 'J Pharmacokinet Pharmacodyn', 'TI': 'Sample size calculation in bioequivalence trials.', 'UI': '22182241', 'VI': '29',

The Python client library returns Python objects, that use [dictionaries](https://docs.python.org/3.6/tutorial/datastructures.html#dictionaries) and [lists](https://docs.python.org/3.6/tutorial/introduction.html#lists).
Use a [for loop](https://docs.python.org/3.6/tutorial/controlflow.html#for-statements) to inspect each hit, and print the retrieved document's titles one by one. 

In [11]:
#THIS IS GRADED!

# your code below
# Get the list of 10,000 hits.
hits = res_10k['hits']['hits']

print("Number of query results returned =", len(hits), "\n\nTitles:")
for i,hit in enumerate(hits):
    print(i, ":", hit['_source']['TI'])

Number of query results returned = 10000 

Titles:
0 : Thrombin functions during tissue factor-induced blood coagulation.
1 : Short deletion within the blood group Dombrock locus causing a Do(null) phenotype.
2 : Working Group on Blood Pressure Monitoring of the European Society of Hypertension International Protocol for validation of blood pressure measuring devices in adults.
3 : Clotting in whole blood: analysis of a biochemical reaction network.
4 : DNB: a partial D with anti-D frequent in Central Europe.
5 : Intrinsic pathway of blood coagulation contributes to thrombogenicity of atherosclerotic plaque.
6 : Persistence of HTLV-I in blood components after leukocyte depletion.
7 : Transplantation of mobilized peripheral blood cells to HLA-identical siblings with standard-risk leukemia.
8 : Absence of CD47 in protein 4.2-deficient hereditary spherocytosis in man: an interaction between the Rh complex and the band 3 complex.
9 : State of the market for devices for blood pressure measu

1239 : Imatinib induces hematologic and cytogenetic responses in patients with chronic myelogenous leukemia in myeloid blast crisis: results of a phase II study.
1240 : Thrombogenicity of beta 2-glycoprotein I-dependent antiphospholipid antibodies in a photochemically induced thrombosis model in the hamster.
1241 : Congenital afibrinogenemia: first identification of splicing mutations in the fibrinogen Bbeta-chain gene causing activation of cryptic splice sites.
1242 : Heterologous cells cooperate to augment stem cell migration, homing, and engraftment.
1243 : Interleukin-1 blockade does not prevent acute graft-versus-host disease: results of a randomized, double-blind, placebo-controlled trial of interleukin-1 receptor antagonist in allogeneic bone marrow transplantation.
1244 : Antimyeloma efficacy of thalidomide in the SCID-hu model.
1245 : [Selected blood coagulation problems in newborn infants]
1246 : Volume of RBCs, 24
1247 : [CD62p expression in platelet during the preparation c

2406 : Dietary fibre in treatment of diabetes: myth or reality?
2407 : Obesity, smoking, and multiple cardiovascular risk factors in young adult African Americans.
2408 : [Markers of oxidative damage in blood of children with cystic fibrosis]
2409 : Assessing the accuracy of three viral risk models in predicting the outcome of implementing HIV and HCV NAT donor screening in Australia and the implications for future HBV NAT.
2410 : The blood platelet as a model for regulating blood coagulation on cell surfaces and its consequences.
2411 : Suicide and the media. Part I: Reportage in nonfictional media.
2412 : Suicide and the media. Part II: Portrayal in fictional media.
2413 : Suicide and the media. Part III: Theoretical issues.
2414 : Blood extraction from lancet wounds using vacuum combined with skin stretching.
2415 : Safer haemotherapy: the responsibilities of government, transfusion service, blood donors, and physician-users.
2416 : Serum cholesterol affects blood pressure regulatio

4072 : Effect of IVIgG treatment on fetal platelet count, HPA-1a titre and clinical outcome in a case of feto-maternal alloimmune thrombocytopenia.
4073 : Effects of peroxidase on hyperlipidemia in mice.
4074 : Effect of buyang huanwu decoction on platelet activating factor content in arterial blood pre
4075 : Evaluating the relationship between arterial blood pressure changes and indices of pulse oximetric plethysmography.
4076 : Multiple sclerosis: low-frequency temporal blood oxygen level-dependent fluctuations indicate reduced functional connectivity initial results.
4077 : Assessment of sirolimus concentrations in whole blood by high-performance liquid chromatography with ultraviolet detection.
4078 : Changes in middle cerebral artery blood flow after carotid endarterectomy as monitored by transcranial Doppler.
4079 : Clinical significance of blood brain natriuretic peptide level measurement in the detection of heart disease in untreated outpatients: comparison of electrocardiogra

5072 : [Changes in synthesis of nitric oxide, blood levels of ACTH and cortisol in viral hepatitis B]
5073 : Allele-specific replication associated with aneuploidy in blood cells of patients with hematologic malignancies.
5074 : [Blood donation]
5075 : [Vitamin A: blood level and dietetics intake in stunted children and adolescents without hormonal disease]
5076 : Long-term effects of oral estradiol and dydrogesterone on carbohydrate metabolism in postmenopausal women.
5077 : In vivo PIV measurement of red blood cell velocity field in microvessels considering mesentery motion.
5078 : [Informative value of immunologic analysis of blood and ejaculate in diagnosing chronic prostatitis]
5079 : Insulinoma in a patient with tuberous sclerosis: is there an association?
5080 : In vitro studies of the influence polyester materials with a different degree of surface wettability have on blood haematological parameters and coagulation and fibrinolysis system parameters.
5081 : A comparative study 

6238 : "Sausage-string" appearance of arteries and arterioles can be caused by an instability of the blood vessel wall.
6239 : Inhibitory mechanism of costunolide, a sesquiterpene lactone isolated from Laurus nobilis, on blood-ethanol elevation in rats: involvement of inhibition of gastric emptying and increase in gastric juice secretion.
6240 : [Blood pressure determination]
6241 : Intrapituitary adenoviral administration of 7B2 can extend life span and reverse endocrinological deficiencies in 7B2 null mice.
6242 : Oat consumption does not affect resting casual and ambulatory 24-h arterial blood pressure in men with high-normal blood pressure to stage I hypertension.
6243 : Clonal T cell receptor gamma-chain gene rearrangement by PCR-based GeneScan analysis in the skin and blood of patients with parapsoriasis and early-stage mycosis fungoides.
6244 : Lipids and nitric oxide in porcine retinal and choroidal blood vessels.
6245 : Mice deficient in the insulin-regulated membrane aminopep

7738 : Peroxidation of proteins and lipids in suspensions of liposomes, in blood serum, and in mouse myeloma cells.
7739 : [The distribution of serum homocysteine and its associated factors in a population of 1 168 subjects in Beijing area]
7740 : Miniaturized electrophoresis: an evolving role in laboratory medicine.
7741 : Reticulocytes. Their usefulness and measurement in peripheral blood.
7742 : Cell saver for blood conservation in cardiac surgery.
7743 : Blood dendritic cells interact with splenic marginal zone B cells to initiate T-independent immune responses.
7744 : Insulin therapy in type 2 diabetes.
7745 : Cerebral blood perfusion after treatment with zolpidem and flumazenil in the baboon.
7746 : [Necessary harmonization of health cost assessment. Autologous peripheral blood progenitor cell transplantation in France]
7747 : Systemic vs. local cytokine and leukocyte responses to unilateral wrist flexion exercise.
7748 : The effect of estrogen use on levels of glucose and insuli

9238 : Multiple organ failure in patients with cardiogenic shock is associated with high plasma levels of interleukin-6.
9239 : Skeletal muscle capillary hemodynamics from rest to contractions: implications for oxygen transfer.
9240 : Interactions between stress, interleukin-1beta, interleukin-6 and cortisol in periodontally diseased patients.
9241 : Is transcranial Doppler ultrasonography (TCD) good enough in determining CO2 reactivity and pressure autoregulation in head-injured patients?
9242 : [Clinical research of patients with acute or chronic hepatic failure treated with molecular adsorbent recirculating system]
9243 : Comparison of PET with radioactive microspheres to assess pulmonary blood flow.
9244 : Bell-bottom aortoiliac endografts: an alternative that preserves pelvic blood flow.
9245 : HEPC-based liposomes trigger cytokine release from peripheral blood cells: effects of liposomal size, dose and lipid composition.
9246 : A simplified double-injection method to quantify cer

## Exercise 02.C: _Search using the Elasticsearch DSL_

You will notice that the native query format of Elasticsearch can be quite verbose.
Elasticsearh provides the Python library `elasticsearch_dsl` to write more concise Elasticsearch queries. 
This is only to simplify the syntax: the library still issues Elasticsearch queries.

For example, a simple `match_all` query looks as follows:
```python
query = {
   "query": {
       "match_all": {}
   }
}
```

The same query can be created with the DSL as follows:
```python
query = Q("match_all")
```

Especially for more complicated boolean queries, to use the native query format can become complicated.
Read more about the DSL [here](https://elasticsearch-dsl.readthedocs.io/en/latest/search_dsl.html)

__Exercise:__ Search for the query `blood` and check whether you get the same number of results as for exercise 02.B. You can use DSL, curl or Kibana. 

In [49]:
#THIS IS GRADED!

# your code here

# BEGIN ANSWER
# Install elasticsearch_dsl library when first running cell.
# ! pip3 install elasticsearch-dsl

import elasticsearch
from elasticsearch_dsl import Search, Q

s = Search(using=es2, index='genomics') 
q = Q('query_string', query='blood')
s = s.query(q)

# Execute the query. This will return the first 10 matches.
res = s.execute()

# Get the total count of matches. 
s.count()  # 68,275 => matches the previous answers.

# END ANSWER

68275

# Making your own TREC run

We will adopt a scientific approach to building search engines. That is, we are not only going to build a search engine and see that it works, but we are also going to _measure_ how well it works, by measuring the search engine's quality. We will adopt the method from the [Text Retrieval Conference](http://trec.nist.gov) (TREC). TREC provides researchers with test collections, that consists of 3 parts:

1. the document collection (in our case a part of the MEDLINE database)
2. the topics (which are natural language descriptions of what the user is searching for: you can think of the as the _queries_)
3. the relevance judgments (for each topic, what documents are relevant)

##  Exercise 02.D

Complete the code of the Python function `make_trec_run()` that reads the topics [training-queries-simple.txt](http:training-queries-simple.txt), and for each topic does a search using Elasticsearch. The program should output a file in the [TREC submission format](https://trec-core.github.io/2017/#submission-guidelines). We already provided the first  lines for this exercise, which include:

1. Open the file `'run_file_name'`' for writing and call it `run_file`.
2. Open the file `'topics_file_name'` for reading, call it `test_queries`.
3. For each line in `test_queries`:
4. Remove the newline using `strip()`, then split the string on the tab character (`'\t'`). The first part of the line is now `qid` (the query identifier) and the last part is `query` (a textual description of the query).
5. complete the Python program such that the correct TREC run file is written to `'run_file_name'`.

> **Note**: Make sure you output the `PMID` (pubmed identifier) of the document `hit['_source']['PMID']`. Do **not** use the elasticsearch identifier `_id` because they do not match the document identifiers in the relevance judgements. They were randomly generated by Elasticsearch during indexing.

In [101]:
#THIS IS GRADED!

import elasticsearch
from elasticsearch_dsl import Search, Q
import re

def make_trec_run(es, topics_file_name, run_file_name, run_name="test"):
    with open(run_file_name, 'w') as run_file:
        with open(topics_file_name, 'r') as test_queries:
            for line in test_queries:
                (qid, query) = line.strip().split('\t')
                # BEGIN ANSWER
                run_tag = "27" + run_name #per TREC guidelines: "each run should have a different tag that identifies the group and the method that produced the run."
                
                # Do search using query. Return 1,000 relevant documents as per question instructions.
                s = Search(using=es2, index='genomics') 
                q = Q('multi_match', query=query)
                s = s.query(q)[0:1000]
                res = s.execute()
              
                for i, hit in enumerate(res.hits.hits):
                    output = [str(qid), 'Q0', str(hit['_source']['PMID']), str(i), str(hit['_score']),run_tag]
                    # Write to file.
                    if i == 0 and qid == 1:
                        run_file.write(" ".join(output))
                    else:
                        run_file.write("\n" + " ".join(output))
                # END ANSWER
                
                
# Write the results of the queries contained in the topic file `'data/training-queries-simple.txt'` 
# to the run file `'baseline.run'`, and name this test as `test01`
make_trec_run(es, 'data/training-queries-simple.txt', 'baseline.run', run_name='test01')

In [102]:
# this prints out (it is a shell command) the content of the file baseline.run 
!cat baseline.run


1 Q0 12056822 0 45.120773 27test01
1 Q0 11929828 1 43.71542 27test01
1 Q0 11943869 2 43.64373 27test01
1 Q0 11751903 3 43.54453 27test01
1 Q0 12573582 4 42.216682 27test01
1 Q0 12384701 5 41.85428 27test01
1 Q0 12624599 6 41.735706 27test01
1 Q0 11981756 7 41.24557 27test01
1 Q0 12175534 8 41.18186 27test01
1 Q0 12065641 9 41.059303 27test01
1 Q0 12444543 10 40.73954 27test01
1 Q0 11933076 11 40.64125 27test01
1 Q0 11980715 12 40.15782 27test01
1 Q0 11790141 13 40.05864 27test01
1 Q0 12018448 14 39.329933 27test01
1 Q0 11876550 15 39.327446 27test01
1 Q0 11759294 16 39.00269 27test01
1 Q0 12126481 17 38.77115 27test01
1 Q0 12045203 18 38.708294 27test01
1 Q0 12036924 19 38.5743 27test01
1 Q0 12052868 20 38.211033 27test01
1 Q0 12012324 21 37.90165 27test01
1 Q0 12214857 22 37.866325 27test01
1 Q0 11886382 23 37.722996 27test01
1 Q0 12429914 24 37.42054 27test01
1 Q0 12455049 25 37.28813 27test01
1 Q0 12444545 26 36.786613 27test01
1 Q0 11880176 27 36.773304 27test01
1 Q0 12081329 28 3

3 Q0 12298013 636 11.10005 27test01
3 Q0 12036988 637 11.098272 27test01
3 Q0 12584999 638 11.089694 27test01
3 Q0 12165438 639 11.087658 27test01
3 Q0 11773055 640 11.085541 27test01
3 Q0 12111492 641 11.082467 27test01
3 Q0 12086825 642 11.078472 27test01
3 Q0 12207604 643 11.07015 27test01
3 Q0 12016971 644 11.063361 27test01
3 Q0 11862618 645 11.058595 27test01
3 Q0 12048686 646 11.058595 27test01
3 Q0 11920742 647 11.049435 27test01
3 Q0 12067473 648 11.048596 27test01
3 Q0 12444553 649 11.048554 27test01
3 Q0 11890680 650 11.048151 27test01
3 Q0 11749167 651 11.04758 27test01
3 Q0 12530267 652 11.04758 27test01
3 Q0 12499504 653 11.04624 27test01
3 Q0 11802805 654 11.038783 27test01
3 Q0 12209017 655 11.035579 27test01
3 Q0 11870317 656 11.03269 27test01
3 Q0 11863072 657 11.030711 27test01
3 Q0 11906605 658 11.030711 27test01
3 Q0 12537542 659 11.030711 27test01
3 Q0 12171855 660 11.027457 27test01
3 Q0 11798890 661 11.025495 27test01
3 Q0 12595689 662 11.021872 27test01
3 Q0 12

6 Q0 12184756 891 17.019085 27test01
6 Q0 11823067 892 17.01178 27test01
6 Q0 12093377 893 17.010807 27test01
6 Q0 12110052 894 17.003908 27test01
6 Q0 12584329 895 17.003891 27test01
6 Q0 12126629 896 17.003885 27test01
6 Q0 11851891 897 16.996159 27test01
6 Q0 11862419 898 16.988703 27test01
6 Q0 12136334 899 16.973583 27test01
6 Q0 12197571 900 16.973583 27test01
6 Q0 11983150 901 16.971012 27test01
6 Q0 11772516 902 16.949112 27test01
6 Q0 12126589 903 16.949112 27test01
6 Q0 12177415 904 16.949112 27test01
6 Q0 12032624 905 16.949112 27test01
6 Q0 12209134 906 16.949112 27test01
6 Q0 11814117 907 16.93686 27test01
6 Q0 11923081 908 16.935795 27test01
6 Q0 11934871 909 16.930727 27test01
6 Q0 11754734 910 16.923122 27test01
6 Q0 11964294 911 16.899706 27test01
6 Q0 11896445 912 16.896275 27test01
6 Q0 12235214 913 16.891325 27test01
6 Q0 12493643 914 16.891212 27test01
6 Q0 12045261 915 16.880068 27test01
6 Q0 12610304 916 16.86244 27test01
6 Q0 12438341 917 16.8511 27test01
6 Q0 1

10 Q0 11733510 226 14.6220665 27test01
10 Q0 12166926 227 14.6153345 27test01
10 Q0 12073176 228 14.614378 27test01
10 Q0 12188900 229 14.602366 27test01
10 Q0 12389889 230 14.594231 27test01
10 Q0 11886590 231 14.569471 27test01
10 Q0 12403827 232 14.56802 27test01
10 Q0 11876568 233 14.564264 27test01
10 Q0 11971762 234 14.5583 27test01
10 Q0 12393857 235 14.5583 27test01
10 Q0 12399449 236 14.5583 27test01
10 Q0 12574114 237 14.5583 27test01
10 Q0 12591740 238 14.5583 27test01
10 Q0 11751295 239 14.5433235 27test01
10 Q0 12066721 240 14.534412 27test01
10 Q0 12442271 241 14.508585 27test01
10 Q0 12227020 242 14.506424 27test01
10 Q0 12360930 243 14.498153 27test01
10 Q0 12522115 244 14.491463 27test01
10 Q0 12153068 245 14.46781 27test01
10 Q0 12175618 246 14.435996 27test01
10 Q0 11802563 247 14.423152 27test01
10 Q0 11802564 248 14.423152 27test01
10 Q0 11802565 249 14.423152 27test01
10 Q0 11802566 250 14.423152 27test01
10 Q0 11802567 251 14.423152 27test01
10 Q0 11802568 252 14

13 Q0 12110274 593 9.497073 27test01
13 Q0 12110275 594 9.497073 27test01
13 Q0 12110276 595 9.497073 27test01
13 Q0 12110277 596 9.497073 27test01
13 Q0 12110278 597 9.497073 27test01
13 Q0 12110279 598 9.497073 27test01
13 Q0 12110280 599 9.497073 27test01
13 Q0 12110281 600 9.497073 27test01
13 Q0 12110282 601 9.497073 27test01
13 Q0 12110283 602 9.497073 27test01
13 Q0 12110284 603 9.497073 27test01
13 Q0 12110285 604 9.497073 27test01
13 Q0 12110286 605 9.497073 27test01
13 Q0 12110287 606 9.497073 27test01
13 Q0 12110288 607 9.497073 27test01
13 Q0 12110289 608 9.497073 27test01
13 Q0 12208857 609 9.496551 27test01
13 Q0 11744393 610 9.496444 27test01
13 Q0 12351678 611 9.495577 27test01
13 Q0 12208849 612 9.493578 27test01
13 Q0 11953451 613 9.490713 27test01
13 Q0 12123598 614 9.488057 27test01
13 Q0 11933094 615 9.486668 27test01
13 Q0 12065414 616 9.481211 27test01
13 Q0 12196028 617 9.476505 27test01
13 Q0 11744384 618 9.472076 27test01
13 Q0 12128227 619 9.472076 27test01
1

...

36 Q0 12144693 659 12.535817 27test01
36 Q0 11929872 660 12.535817 27test01
36 Q0 11959829 661 12.535817 27test01
36 Q0 12050657 662 12.535817 27test01
36 Q0 12021767 663 12.535817 27test01
36 Q0 11972041 664 12.535817 27test01
36 Q0 11734896 665 12.527788 27test01
36 Q0 12194025 666 12.527788 27test01
36 Q0 12455976 667 12.527788 27test01
36 Q0 12471248 668 12.514529 27test01
36 Q0 11932766 669 12.500429 27test01
36 Q0 12502789 670 12.500429 27test01
36 Q0 11839821 671 12.498127 27test01
36 Q0 12520017 672 12.488462 27test01
36 Q0 12520032 673 12.488462 27test01
36 Q0 11765131 674 12.486958 27test01
36 Q0 11902679 675 12.486958 27test01
36 Q0 11744370 676 12.480515 27test01
36 Q0 11925450 677 12.456507 27test01
36 Q0 11919704 678 12.429957 27test01
36 Q0 11781497 679 12.3741255 27test01
36 Q0 11835058 680 12.3741255 27test01
36 Q0 11880339 681 12.3741255 27test01
36 Q0 11927561 682 12.3741255 27test01
36 Q0 11928503 683 12.3741255 27test01
36 Q0 12095613 684 12.3741255 27test01
36 Q0 

39 Q0 11951023 928 11.048149 27test01
39 Q0 12466193 929 11.048149 27test01
39 Q0 11914715 930 11.005099 27test01
39 Q0 11926534 931 11.005099 27test01
39 Q0 11937021 932 11.005099 27test01
39 Q0 11948621 933 11.005099 27test01
39 Q0 12465973 934 11.005099 27test01
39 Q0 12111720 935 11.005099 27test01
39 Q0 12372142 936 11.005099 27test01
39 Q0 12374572 937 11.005099 27test01
39 Q0 12459439 938 10.999022 27test01
39 Q0 12522919 939 10.966305 27test01
39 Q0 12296541 940 10.955241 27test01
39 Q0 11997349 941 10.931357 27test01
39 Q0 12242050 942 10.925026 27test01
39 Q0 11836794 943 10.887803 27test01
39 Q0 12586062 944 10.887803 27test01
39 Q0 11875653 945 10.871458 27test01
39 Q0 12183374 946 10.871458 27test01
39 Q0 12186640 947 10.871458 27test01
39 Q0 12200166 948 10.871458 27test01
39 Q0 12208494 949 10.871458 27test01
39 Q0 12505988 950 10.871458 27test01
39 Q0 12111742 951 10.871458 27test01
39 Q0 12372288 952 10.871458 27test01
39 Q0 12454022 953 10.871458 27test01
39 Q0 125355

43 Q0 11961101 204 15.279039 27test01
43 Q0 12520013 205 15.273477 27test01
43 Q0 11881810 206 15.242591 27test01
43 Q0 12035802 207 15.239593 27test01
43 Q0 12447381 208 15.234868 27test01
43 Q0 12136413 209 15.231316 27test01
43 Q0 12072176 210 15.226326 27test01
43 Q0 12436113 211 15.210492 27test01
43 Q0 12054512 212 15.195192 27test01
43 Q0 12006672 213 15.184298 27test01
43 Q0 11904380 214 15.177974 27test01
43 Q0 11914720 215 15.177974 27test01
43 Q0 12134149 216 15.177974 27test01
43 Q0 12141779 217 15.177974 27test01
43 Q0 12183041 218 15.177974 27test01
43 Q0 12007414 219 15.177974 27test01
43 Q0 12210559 220 15.177974 27test01
43 Q0 12486701 221 15.177974 27test01
43 Q0 12350270 222 15.177974 27test01
43 Q0 12398416 223 15.177974 27test01
43 Q0 12559957 224 15.177974 27test01
43 Q0 12093382 225 15.145387 27test01
43 Q0 12139511 226 15.133473 27test01
43 Q0 11989718 227 15.112351 27test01
43 Q0 12144703 228 15.10177 27test01
43 Q0 11951037 229 15.096924 27test01
43 Q0 1208211

46 Q0 11948614 114 14.251796 27test01
46 Q0 12004963 115 14.240157 27test01
46 Q0 12438220 116 14.240157 27test01
46 Q0 12555612 117 14.216894 27test01
46 Q0 12418572 118 14.194429 27test01
46 Q0 11855817 119 14.154394 27test01
46 Q0 12442833 120 14.139578 27test01
46 Q0 11803384 121 14.076817 27test01
46 Q0 12015302 122 14.054104 27test01
46 Q0 12110583 123 14.054104 27test01
46 Q0 12553307 124 14.032275 27test01
46 Q0 12480532 125 13.966044 27test01
46 Q0 11723434 126 13.895734 27test01
46 Q0 11913791 127 13.865978 27test01
46 Q0 12362432 128 13.865163 27test01
46 Q0 12063404 129 13.858891 27test01
46 Q0 12203814 130 13.849791 27test01
46 Q0 12070181 131 13.843348 27test01
46 Q0 12298002 132 13.821615 27test01
46 Q0 12112321 133 13.76326 27test01
46 Q0 12370331 134 13.5883875 27test01
46 Q0 12489150 135 13.545003 27test01
46 Q0 12093749 136 13.436757 27test01
46 Q0 11935223 137 13.424397 27test01
46 Q0 12500631 138 13.416143 27test01
46 Q0 12507466 139 13.412858 27test01
46 Q0 123567

49 Q0 12086873 794 9.384027 27test01
49 Q0 12349953 795 9.383721 27test01
49 Q0 11960378 796 9.380508 27test01
49 Q0 12097382 797 9.378643 27test01
49 Q0 12207231 798 9.378643 27test01
49 Q0 11846978 799 9.373284 27test01
49 Q0 12163407 800 9.373142 27test01
49 Q0 11738799 801 9.371589 27test01
49 Q0 12223398 802 9.370614 27test01
49 Q0 12210033 803 9.36909 27test01
49 Q0 12580964 804 9.368754 27test01
49 Q0 12049780 805 9.3642025 27test01
49 Q0 12392794 806 9.3642025 27test01
49 Q0 12586340 807 9.3642025 27test01
49 Q0 12517926 808 9.363464 27test01
49 Q0 12388793 809 9.355129 27test01
49 Q0 12438606 810 9.355129 27test01
49 Q0 12230120 811 9.354106 27test01
49 Q0 12130544 812 9.353748 27test01
49 Q0 12089160 813 9.352926 27test01
49 Q0 12223476 814 9.352579 27test01
49 Q0 12483323 815 9.351791 27test01
49 Q0 12079879 816 9.349871 27test01
49 Q0 12213324 817 9.348341 27test01
49 Q0 12154401 818 9.345437 27test01
49 Q0 11876909 819 9.341505 27test01
49 Q0 11971180 820 9.341505 27test01

> Tip: Write a line to `run_file` using `run_file.write(line)`. 
> The newline character is: `'\n'`. Before writing a number to
> the file, cast it to a string using `str()`.
>
> The TREC Submission guidelines allow you to submit up to 1000
> documents per topic. Keep this in mind!

# Index improvements: Tokenization (Analyzers)

_You are advised to work on this part after Lecture 02_

The way documents are indexed influences the performance of the IR systems. 
Elasticsearch [Mappings](https://www.elastic.co/guide/en/elasticsearch/reference/6.2/mapping.html) define how a document, and its properties (fields) are stored and indexed. When using a different configuration of an ElasticSearch Mapping, the document collection needs to be re-indexed.

## Background
The following part of the assignment requires some self-study of the ElasticSearch tools to support the improvemnet of the indexing. Please read the:
* [Index Settings and Mappings](https://www.elastic.co/guide/en/elasticsearch/reference/6.2/indices-create-index.html).
* Elasticsearch [Analyzers](https://www.elastic.co/guide/en/elasticsearch/reference/6.2/analysis.html) contain many options for improving your search engine.

You are again requested to use the [Python Elasticsearch Client](https://elasticsearch-py.readthedocs.io) library documentation.


## Bulk indexing revisited
As we need to re-index the document collection when we use a different Mapping configuration, we developed some functions to support a quick re-indexing in the following exercises.

Below you find the Python code for bulk-indexing our Medline collection, similar to the code of the exercises in the beginning of Part 02. Read the code carefully, as you are required to use the indexing functions later for the completion of the assignment.

> The code uses additional helper functions 
> (`elasticsearch.helpers`) and a library for processing JSON.
> The function `read_documents()` reads the bulk insert file: The 
> function is a generator function. It generates an 'on-demand' list
> by using the statement `yield` for every item of the list. It
> is used in the helper function `elasticsearch.helpers.bulk()`.
> The statement `raise` is Python's approach to throw exceptions, 
> that is, it exits the program with an error.
> Note the additional (keyword) arguments to bulk:
> `chunk_size` indicates the number of documents to be processed by
> elasticsearch in one batch. 
> The request_timeout is set to 30 seconds because processing a single batch
> of documents can take some time.

> __Note:__ _when processing a bulk index, be sure to have few GigaBytes free on your hard drive. If you get a BulkIndexError with read-only/FORBIDDEN errors, you probably have too little hard drive space available for ElasticSearch to work properly._


In [1]:
import elasticsearch
import elasticsearch.helpers
import json

es = elasticsearch.Elasticsearch(host='localhost')  # in case you use Docker, the host is 'elasticsearch'

def read_documents(file_name):
    """
    Returns a generator of documents to be indexed by elastic, read from file_name
    """
    with open(file_name, 'r') as documents:
        for line in documents:
            doc_line = json.loads(line)
            if ('index' in doc_line):
                id = doc_line['index']['_id']
            elif ('PMID' in doc_line):
                doc_line['_id'] = id
                yield doc_line
            else:
                raise ValueError('Woops, error in index file')

def create_index(es, index_name, body={}):
    # delete index when it already exists
    es.indices.delete(index=index_name, ignore=[400, 404])
    # create the index 
    es.indices.create(index=index_name, body=body)
                
def index_documents(es, file_name, index_name, body={}):
    create_index(es, index_name, body)
    # bulk index the documents from file_name
    return elasticsearch.helpers.bulk(
        es, 
        read_documents(file_name),
        index=index_name,
        chunk_size=2000,
        request_timeout=30
    )

In [3]:
index_documents(es, 'data/trec-medline.json', 'genomics')

(525937, [])

## Exercise 02.E: _ElasticSearch Analyzers (Tokenization)_

The amount and quality of the tokens used to construct the inverted index are of great importance. In ElasticSearch, mappings and settings also allow specifying what [Analyzer](https://www.elastic.co/guide/en/elasticsearch/reference/current/analysis.html) is used to tokenize your documents and queries. In the mappings below, use the _Dutch_ analyzer for the field `"all"` (where `"all'` indexes the fields `"TI"` and `"AB"`):

> Usually, the same analyzer should be applied to documents and queries, but 
> Elasticsearch allows you to specify a `"search_analyzer"` that is used on 
> your queries (which we do not need to use in the assignment).

In [20]:
#THIS IS GRADED!

analyzer_test = {
  # BEGIN ANSWER
    "mappings": {
        "properties": {
            "all": {
                "type": "text",
                "analyzer": "dutch",
                "fields": {
                    "AB": {
                        "type": "text",
                    },
                    "TI": {
                    "type": "text",
                    }
                }
            }
        }
    }
  # END ANSWER
}

# create the index, but don't index any documents:
create_index(es, 'genomics', body=analyzer_test)

The analyzer defined for the `"all"` field can be tested [as follows](https://elasticsearch-py.readthedocs.io/en/master/api.html#indices). Translated to English the text says: _"This is a Dutch sentence"_. 

    The following script identifies the tokens (based on the use of the dutch tokenizer): try with different tokenizers and different sentences to see how the tokens are created.

In [15]:
from pprint import pprint # pretty print

body = { "field": "all", "text": "dit zijn nederlandse zinnen"}
tokens = es.indices.analyze(index='genomics', body=body)
pprint(tokens)

{'tokens': [{'end_offset': 20,
             'position': 2,
             'start_offset': 9,
             'token': 'nederland',
             'type': '<ALPHANUM>'},
            {'end_offset': 27,
             'position': 3,
             'start_offset': 21,
             'token': 'zinn',
             'type': '<ALPHANUM>'}]}


##  Exercise 02.F: _tweet language analyzer_

Read the documentation for [Custom Analyzer](https://www.elastic.co/guide/en/elasticsearch/reference/6.2/analysis-custom-analyzer.html). 
Make a custom analyzer for _English tweet language_. The analyzer should do the following:
* change common abbreviations to the full forms: 
  * _b4_ to _before_, 
  * _abt_ to _about_, 
  * _chk_ to _check_, 
  * _cr8_ to _create_, 
  * _dm_ to _direct message_,
  * _f2f_ to _face-to-face_
* use the _standard_ tokenizer;
* put everything to lower-case;
* filter English stopwords.

In [32]:
#THIS IS GRADED!

tweet_analyzer = {
  # BEGIN ANSWER
    "mappings": {
        "properties": {
            "all": {
                "type": "text",
                "analyzer": "tweet_analyzer",
                "fields": {
                    "AB": {
                        "type": "text",
                    },
                    "TI": {
                    "type": "text",
                    }
                }   
            }
        }
    },
    "settings": {
        "analysis": {
            "analyzer": {
                 "tweet_analyzer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "char_filter": "tweet_char_filter",
                    "filter": [
                        "lowercase",
                        "stop_filter" 
                    ]
                 }
            },
            "char_filter": {
                "tweet_char_filter": {
                    "type":"mapping",
                    "mappings": [
                        "b4 => before",
                        "abt => about",
                        "chk => check",
                        "cr8 => create",
                        "dm => direct message",
                        "f2f => face-to-face" 
                    ]
                }
            },
            "filter": {
                "stop_filter": {
                    "type": "stop",
                    "stopwords": "_english_"
                }
            }
        }
    }
  # END ANSWER
}

# create the index, but don't index any documents:
create_index(es, 'genomics', body=tweet_analyzer)
body = { "field": "all", "text": "cr8 it! what abt dm me?"}
tokens = es.indices.analyze(index='genomics', body=body)
pprint(tokens)

{'tokens': [{'end_offset': 3,
             'position': 0,
             'start_offset': 0,
             'token': 'create',
             'type': '<ALPHANUM>'},
            {'end_offset': 12,
             'position': 2,
             'start_offset': 8,
             'token': 'what',
             'type': '<ALPHANUM>'},
            {'end_offset': 16,
             'position': 3,
             'start_offset': 13,
             'token': 'about',
             'type': '<ALPHANUM>'},
            {'end_offset': 18,
             'position': 4,
             'start_offset': 17,
             'token': 'direct',
             'type': '<ALPHANUM>'},
            {'end_offset': 19,
             'position': 5,
             'start_offset': 18,
             'token': 'message',
             'type': '<ALPHANUM>'},
            {'end_offset': 22,
             'position': 6,
             'start_offset': 20,
             'token': 'me',
             'type': '<ALPHANUM>'}]}


# Part 03: Search models 

_You are advised to work on this part after Lecture 03_

Elasticsearch [Mappings](https://www.elastic.co/guide/en/elasticsearch/reference/7.8/mapping.html) define how a document, and its properties (fields) are stored and indexed, but also provides tools to implement and exeute different document similarity measures (i.e. search models). 

> See again: [Index Settings and Mappings](https://www.elastic.co/guide/en/elasticsearch/reference/7.8/indices-create-index.html).

For instance, we can add a new field `"all"` that uses the  [similarity measure](https://www.elastic.co/guide/en/elasticsearch/reference/7.8/similarity.html) _Boolean_, and let it serve as an index for the fields `"TI"` and `"AB"` (title and abstract) as follows:

In [ ]:
boolean = {
  "settings" : {
    # a single shard, so we do not suffer from approximate document frequencies
    "number_of_shards" : 1
  },
  "mappings": {
      "properties": {
        "AB": {
          "type": "text",
          "similarity": "boolean"
        },
        "TI": {
          "type": "text",
          "similarity": "boolean"
        }
      }
  }
}

index_documents(es, 'data/trec-medline.json', 'genomics', body=boolean)

> Most changes to the mappings cannot be done on an existing index. Some (for instance
> similarity measures) can be changed if the index is first closed. Nevertheless, we 
> will in this notebook _re-index_ the collection for every change to the mappings
> using the function `index_documents()` that we defined above. Mappings (and settings)
> can be passed to the function using the `body` parameter.

Let's have a look at the mappings and settings for our index as follows:

In [ ]:
es.indices.get(index='genomics')

Now let's search our new field `"all"` as follows:

In [ ]:
query = "blood"
search_type = "dfs_query_then_fetch" # this will use exact document frequencies even for multiple shards
body = {
  "query": {
    "match" : { "all" : query }
  },
  "size": 10
}
es.search(index="genomics", search_type=search_type, body=body)

## Exercise 03.A: _new run and evaluation_
Create a new run file (e.g. `boolean.run`), compute the retrieval performance with the function `print_trec_eval()` and compare the results with the baseline run file `baseline.run`.

In [ ]:
#THIS IS GRADED!

# write your code here
# BEGIN ANSWER
# END ANSWER

## Exercise 03.B: _Language models_

Custom similarities can be configured by tuning the parameters of the built-in similarities. Read more about these (expert) options in the [similarity module](https://www.elastic.co/guide/en/elasticsearch/reference/6.2/index-modules-similarity.html).

> Tip: the example similarity settings have to be used in a `"settings"` object.
> Check your settings and mappings with: `es.indices.get(index='genomics')`.

Make a run that uses Language Models with Jelinek-Mercer smoothing (linear interpolation smoothing) on the field `"all"` that indexes the fields `"TI"` and `"AB"`. Use the parameter `lambda=0.2`. Again evaluate the run using `print_trec_eval`.

In [ ]:
#THIS IS GRADED!

lmjelinekmercer = {
  # BEGIN ANSWER
  # END ANSWER
}

index_documents(es, 'data/trec-medline.json', 'genomics', body=lmjelinekmercer)
make_trec_run(es, 'data/training-queries-simple.txt', 'lmjelinekmercer.run')

## Exercise 03.C: _Model comparison_
Compute the results of the `lmjelinekmercer.run` and compare them with those of the `baseline.run` and `boolean.run`. Performing statistical tests may help strenghtening your claims.

In [ ]:
#THIS IS GRADED!

# your comments here
# BEGIN ANSWER
# END ANSWER

In [ ]:
! head -20 baseline.run
! echo "\n"
! head -20 lmjelinekmercer.run

## Exercise 03.D: _Implement your own similarity measure (Bonus)_ 

We have only seen the results of using the analyzer to queries. The analyzer results from the _documents_ are available using the `termvectors()` function, as follows for document `id=1`: (Additionally, we can get overall field statistics, such as the number of documents)

> First, index the collection again. While waiting, have a coffee or tea :) 

> `id=1` refers to the internal document identifiers, so not to the Pubmed identifier.

_The bonus exercise is not mandatory. It can compensate for lower grades of other exercises._

In [ ]:
index_documents(es, 'data/trec-medline.json', 'genomics')

es.termvectors(index="genomics", id="1", fields="TI", 
               term_statistics=True, field_statistics=True, offsets=False)

### Implement the BM25 similarity

Complete the function `bm25_similarity()` below by implementing the BM25 similarity as described by in Section 11.4.3 of [Manning, Raghavan and Schuetze, Chapter 11](https://nlp.stanford.edu/IR-book/pdf/11prob.pdf). Are you able to replicate the score of ElasitcSearch (15.472)? If not, are you using a different variant of the BM25 model?

In [ ]:
#THIS IS GRADED!

import math

# math.log(x) computes the logarithm of x

def bm25_similarity (query, doc_id):

    # Get the query tokens (see above)
    query_tokens = es.indices.analyze(index='genomics', body={"field":"TI", "text": query})
    tokens = query_tokens['tokens']

    # Get the term vector for doc_id and the field statistics
    term_vector = es.termvectors(index="genomics", id=doc_id, fields="TI", 
                  term_statistics=True, field_statistics=True, offsets=False)
    vector = term_vector['term_vectors']['TI']['terms']
    f_stats = term_vector['term_vectors']['TI']['field_statistics']

    # The answer should sum over 'tokens', check if the tokens exists in the 'vector',
    # and if so, add the appropriate value to 'similarity'.
    # Tip: add print statements to your code to see what each variable contains.
    
    similarity = 0

    # BEGIN ANSWER
    # END ANSWER
    return similarity

bm25_similarity("curve fitting", 1)

See below for the 'reference score' computed by ElasticSearch:

In [ ]:
body = {
  "query": {
    "match" : { "TI" : "curve fitting" }
  }
}
explain = es.explain(index="genomics", id="1", body=body)
print (explain['explanation']['value'])  # BM25 score computed by ElasticSearch